In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/marta/nltk_data...


True

In [6]:
import re
import ast
import time
import nltk
import joblib
import random
import codecs
# import wordnet
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from scipy import stats
from nltk import pos_tag
from random import shuffle
import plotly.express as px
import plotly.subplots as sp
from textblob import TextBlob
from nltk import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from wordcloud import STOPWORDS
from scipy.sparse import hstack
import plotly.graph_objects as go
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
# from gensim.models import Word2Vec
from geopy.geocoders import Nominatim
from nltk.stem import WordNetLemmatizer
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from datasketch import MinHash, MinHashLSHForest
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/marta/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/marta/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/marta/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/marta/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True